In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def rwma(data):
    #data is an N by T array with returns for each expert/stock i in time t forming a row
    #data = np.array(data)
    #data = totalassetdata
    
    N = len(data.columns) # number of stocks
    T = len(data)
    
    eps = np.sqrt(np.log(N)/T)
    
    w = np.ones(N)
    #M = np.zeros(N)
    #M_wma_random = 0

    #allweights = []

    # iterations
    for t in range(T):
               
        # reveal outcome, suppose it is random
        y = (data.iloc[t]<0).astype(int) #find for each expert if return was negative (loss 1)
        
        #M_wma_random += (x != y) #count number of mistakes

        # weight update
        for i in range(N):
            w[i] = w[i]*(1 - eps)**(y[i]) # penalize weight if made wrong prediction
        #M = M + (f != y)
    
    p = w/sum(w)
    
    print(eps,T,N,w)
   
    return p

In [4]:
data = pd.read_excel('trading_data.xlsx')
(rwma(data))

0.05971638133786811 997 35 [3.13133378e-14 1.86733455e-13 5.00126898e-13 1.65097263e-13
 1.21348447e-13 8.38662922e-14 1.49567007e-14 6.01594147e-13
 4.15773516e-13 3.76662820e-14 3.45647510e-13 3.76662820e-14
 8.91925485e-14 4.81856002e-14 2.54055021e-13 7.69605428e-13
 4.42178836e-13 3.67599204e-13 1.75582409e-13 9.25745292e-13
 8.70463133e-13 4.26024930e-14 1.37251305e-13 3.05598468e-13
 1.29055154e-13 2.38883775e-13 7.69605428e-13 2.44773751e-14
 3.67599204e-13 1.91337601e-14 6.01594147e-13 1.65097263e-13
 5.79616398e-14 9.48570694e-14 1.55238252e-13]


1.0

In [5]:
x=rwma(data)
out_dict = {data.columns[i]:round(x[i],5) for i in range(len(data.columns))}
out_dict

0.05971638133786811 997 35 [3.13133378e-14 1.86733455e-13 5.00126898e-13 1.65097263e-13
 1.21348447e-13 8.38662922e-14 1.49567007e-14 6.01594147e-13
 4.15773516e-13 3.76662820e-14 3.45647510e-13 3.76662820e-14
 8.91925485e-14 4.81856002e-14 2.54055021e-13 7.69605428e-13
 4.42178836e-13 3.67599204e-13 1.75582409e-13 9.25745292e-13
 8.70463133e-13 4.26024930e-14 1.37251305e-13 3.05598468e-13
 1.29055154e-13 2.38883775e-13 7.69605428e-13 2.44773751e-14
 3.67599204e-13 1.91337601e-14 6.01594147e-13 1.65097263e-13
 5.79616398e-14 9.48570694e-14 1.55238252e-13]


{'XOM': 0.00326,
 'SBUX': 0.01946,
 'AAPL': 0.05213,
 'GOOG': 0.01721,
 'PYPL': 0.01265,
 'FB': 0.00874,
 'NFLX': 0.00156,
 'SQ': 0.06271,
 'MSFT': 0.04334,
 'IBM': 0.00393,
 'MCD': 0.03603,
 'AMZN': 0.00393,
 'NKE': 0.0093,
 'TSLA': 0.00502,
 'EL': 0.02648,
 'CSCO': 0.08022,
 'WMT': 0.04609,
 'KO': 0.03832,
 'PM': 0.0183,
 'PG': 0.0965,
 'PEP': 0.09074,
 'GS': 0.00444,
 'BAC': 0.01431,
 'JPM': 0.03186,
 'BLK': 0.01345,
 'ICE': 0.0249,
 'SO': 0.08022,
 'PBR': 0.00255,
 'RUN': 0.03832,
 'MRO': 0.00199,
 'PFE': 0.06271,
 'ANTM': 0.01721,
 'CI': 0.00604,
 'HUM': 0.00989,
 'JNJ': 0.01618}